## Amazon Hackathon -- Product Recommendation

#### Jason Wu  -- 2022/10/20

Amazon collaborated with Columbia University together holding a Hackathon with a theme of recommending products (Dog food). This project builds a model using Neural Network and predict the probabilities for each dog food brand that a new Amazon user may potentially purchase.

## Part 1: Import Package & Dataset

In [1]:
#Import package
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [2]:
raw_df = pd.read_excel('https://github.com/JasonWu414/Amazon_Hackathon_Proteam/blob/main/Columbia_Hackathon_Data_Dogfood.xlsx?raw=true')
raw_df.head()

,sale_id,sale_date,ad_exp,sns,product_id,product_brand,product_name,price,qty,customer_id,...,city,st,zip,lat,lng,marital,education,income,age,prime
0,1,2021-10-01,Don't recall seeing an ad,0,3,Alpha,"Alpha Natural Sensitive Systems, Skin & Coat S...",48.99,1,370,...,Boise,ID,83711,43.4599,-116.2440,Married,Some college or trade school,"$80,000 - $99,999",55-64,0
1,2,2021-10-01,Don't recall seeing an ad,1,14,Arf,Arf Soft & Tender American Jerky Dog Treats,11.52,1,625,...,Durham,NC,27710,36.0512,-78.8577,Married,High school graduate,"$100,000 or more",45-54,1
2,3,2021-10-01,Don't recall seeing an ad,0,23,Bezt,Bezt Adult Chicken and Brown Rice Recipe Dry D...,14.22,1,905,...,Phoenix,AZ,85099,33.2765,-112.1872,Married,College graduate,"$100,000 or more",45-54,1
3,4,2021-10-01,Don't recall seeing an ad,0,33,Alpha,"Alpha Probiotics Shredded Blend High Protein, ...",79.31,1,1061,...,Portsmouth,NH,214,43.0059,-71.0132,Married,Some college or trade school,"$100,000 or more",55-64,1
4,5,2021-10-01,Don't recall seeing an ad,0,26,Alpha,Alpha Natural Adult Lamb & Rice Dry Dog Food,25.16,1,1411,...,Chicago,IL,60624,41.8804,-87.7223,Single,Some college or trade school,"$40,000 - $59,999",25-34,1


## Part 2: EDA

##### in here, I does not create any charts because I use tableau to create them, and all the charts are in the file "Slides" (Also in the Github)

In [3]:
print(raw_df.shape)
print("-----------------------------------------")
print(raw_df.columns)
print("-----------------------------------------")
print(raw_df.loc[:,"sns"].value_counts())
print("-----------------------------------------")
print(raw_df.loc[:,"qty"].value_counts())
print("-----------------------------------------")
print(raw_df.loc[:,"product_name"].value_counts())
print("-----------------------------------------")
print(raw_df.nunique())
print("-----------------------------------------")
print(raw_df.isnull().sum())
print("-----------------------------------------")
print(raw_df.loc[:,"ad_exp"].value_counts())

(8894, 21)
-----------------------------------------
Index(['sale_id', 'sale_date', 'ad_exp', 'sns', 'product_id', 'product_brand',
       'product_name', 'price', 'qty', 'customer_id', 'gender', 'city', 'st',
       'zip', 'lat', 'lng', 'marital', 'education', 'income', 'age', 'prime'],
      dtype='object')
-----------------------------------------
0    6698
1    2196
Name: sns, dtype: int64
-----------------------------------------
1    8182
2     681
3      31
Name: qty, dtype: int64
-----------------------------------------
Alpha Natural Healthy Weight Formula Adult Dry Dog Food & Wet Dog Food    524
Alpha Sensitive Skin & Stomach, High Protein Dry Dog Food                 362
Alpha Natural Adult Chicken & Rice Dry Dog Food                           340
Alpha Natural Sensitive Systems, Skin & Coat Salmon Adult Dry Dog Food    303
Bezt Natural Adult Small Breed Dry Dog Food                               288
                                                                         ..

## Part 3: Feature engineering

In [4]:
#base on the results from tableau EDA, we find that we can use the longtitude of the Kansas 
filtered_df = raw_df.copy()
filtered_df.loc[(filtered_df['lng'] >= -98.5795), 'Geography'] = "Eastern"
filtered_df.loc[(filtered_df['lng'] < -98.5795), 'Geography'] = "Western"
filtered_df.head(3)

,sale_id,sale_date,ad_exp,sns,product_id,product_brand,product_name,price,qty,customer_id,...,st,zip,lat,lng,marital,education,income,age,prime,Geography
0,1,2021-10-01,Don't recall seeing an ad,0,3,Alpha,"Alpha Natural Sensitive Systems, Skin & Coat S...",48.99,1,370,...,ID,83711,43.4599,-116.2440,Married,Some college or trade school,"$80,000 - $99,999",55-64,0,Western
1,2,2021-10-01,Don't recall seeing an ad,1,14,Arf,Arf Soft & Tender American Jerky Dog Treats,11.52,1,625,...,NC,27710,36.0512,-78.8577,Married,High school graduate,"$100,000 or more",45-54,1,Eastern
2,3,2021-10-01,Don't recall seeing an ad,0,23,Bezt,Bezt Adult Chicken and Brown Rice Recipe Dry D...,14.22,1,905,...,AZ,85099,33.2765,-112.1872,Married,College graduate,"$100,000 or more",45-54,1,Western


In [5]:
filtered_df.columns

Index(['sale_id', 'sale_date', 'ad_exp', 'sns', 'product_id', 'product_brand',
       'product_name', 'price', 'qty', 'customer_id', 'gender', 'city', 'st',
       'zip', 'lat', 'lng', 'marital', 'education', 'income', 'age', 'prime',
       'Geography'],
      dtype='object')

In [6]:
#drop useless column
filtered_df = filtered_df.drop(columns=['sale_id','sale_date', 'product_id','customer_id','zip','product_name','st','city','lat','lng'])
cleaned_df = filtered_df.copy()

In [7]:
filtered_df.head(3)

,ad_exp,sns,product_brand,price,qty,gender,marital,education,income,age,prime,Geography
0,Don't recall seeing an ad,0,Alpha,48.99,1,F,Married,Some college or trade school,"$80,000 - $99,999",55-64,0,Western
1,Don't recall seeing an ad,1,Arf,11.52,1,F,Married,High school graduate,"$100,000 or more",45-54,1,Eastern
2,Don't recall seeing an ad,0,Bezt,14.22,1,M,Married,College graduate,"$100,000 or more",45-54,1,Western


#### ONE_HOT_ENCODING

In [8]:
#ONE_HOT_ENCODING
def OneHotEncoding(df, enc, categories):  
  transformed = pd.DataFrame(enc.transform(df[categories]).toarray(), columns=enc.get_feature_names(categories))
  return pd.concat([df.reset_index(drop=True), transformed], axis=1).drop(categories, axis=1)

categories = ['ad_exp','gender','marital','education','income','age',"Geography"]
enc_ohe = OneHotEncoder()
enc_ohe.fit(filtered_df[categories])

filtered_df = OneHotEncoding(filtered_df, enc_ohe, categories)


#Label Encoder Dict
le = LabelEncoder()
ids = le.fit_transform(filtered_df['product_brand'])
filtered_df['product_brand'] = ids
mapping = dict(zip(le.classes_, range(len(le.classes_))))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#### Standarization

In [9]:
scaler = StandardScaler()
scaler.fit(filtered_df[["price"]])
filtered_df[["price"]] = scaler.transform(filtered_df[["price"]])
filtered_df.head(3)
filtered_df.shape

(8894, 34)

## Part 4: Building model

In [10]:
X_train, X_validation, Y_train, Y_validation = train_test_split(filtered_df.drop(['product_brand'], axis=1), filtered_df['product_brand'], test_size=0.20, random_state=0, stratify=filtered_df['product_brand'])

In [11]:

# Convert to array values
# Only the values in the DataFrame will be returned, the axes labels will be removed
X_train = X_train.values
Y_train = Y_train.values
X_validation = X_validation.values
Y_validation = Y_validation.values

# save the non-one-hot encoded version for plotting confusion matrix
Y_validation_confusion = Y_validation

Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=25)
Y_validation = tf.keras.utils.to_categorical(Y_validation, num_classes=25)
print(Y_validation.shape)

(1779, 25)


In [12]:
def baseline_model():
    model = Sequential()
    model.add(Dense(33, input_dim = 33, activation = 'relu'))
    model.add(Dense(28, activation = 'relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(25, activation = 'softmax'))
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [13]:
estimator = KerasClassifier(build_fn = baseline_model, epochs = 100, batch_size = 20)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)
results = cross_val_score(estimator, X_train, Y_train, cv = kfold)
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


Epoch 1/100
285/285 [==============================] - 3s 3ms/step - loss: 2.5586 - accuracy: 0.2902
Epoch 2/100
285/285 [==============================] - 1s 3ms/step - loss: 2.2573 - accuracy: 0.3403
Epoch 3/100
285/285 [==============================] - 1s 3ms/step - loss: 2.1703 - accuracy: 0.3410
Epoch 4/100
285/285 [==============================] - 1s 3ms/step - loss: 2.1204 - accuracy: 0.3398
Epoch 5/100
285/285 [==============================] - 1s 3ms/step - loss: 2.0689 - accuracy: 0.3452
Epoch 6/100
285/285 [==============================] - 1s 3ms/step - loss: 2.0442 - accuracy: 0.3475
Epoch 7/100
285/285 [==============================] - 1s 3ms/step - loss: 2.0257 - accuracy: 0.3405
Epoch 8/100
285/285 [==============================] - 1s 3ms/step - loss: 2.0054 - accuracy: 0.3493
Epoch 9/100
285/285 [==============================] - 1s 3ms/step - loss: 1.9820 - accuracy: 0.3501
Epoch 10/100
285/285 [==============================] - 1s 3ms/step - loss: 1.9772 - accura

In [14]:
model = baseline_model()
model.summary()
print(Y_validation.shape)
fitted_model = model.fit(X_train, Y_train, validation_data=(X_validation, Y_validation), epochs=100, batch_size=20)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 33)                1122      
                                                                 
 dense_16 (Dense)            (None, 28)                952       
                                                                 
 dropout_5 (Dropout)         (None, 28)                0         
                                                                 
 dense_17 (Dense)            (None, 25)                725       
                                                                 
Total params: 2,799
Trainable params: 2,799
Non-trainable params: 0
_________________________________________________________________
(1779, 25)
Epoch 1/100
356/356 [==============================] - 2s 4ms/step - loss: 2.5007 - accuracy: 0.3065 - val_loss: 2.2127 - val_accuracy: 0.3502
Epoch 2/100
356/356 [=========

In [15]:
predict_y = model.predict(X_validation) 
classes_y = np.argmax(predict_y,axis=1)
print(classes_y.shape)
print(classes_y)

56/56 [==============================] - 0s 1ms/step
(1779,)
[ 5  0  5 ...  1 11  5]


In [16]:
Deliver_output = pd.DataFrame(predict_y, columns = ['Alpha','Arf','Astro','Beam','Beethoven','Bezt','Bones','Choice','Flora','Garland','Hanover','Health One', 'Hearth','K99','Kastle','King','Omaha','Paws','Perro','Playtime','Rivera','Romero','Ruby','Seattle Gourmet','Top'])
Deliver_output = Deliver_output.round(3)
sample_output = Deliver_output.head(3)
sample_output["Customer"] = [1,2,3]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [17]:
first_column = sample_output.pop('Customer')
  
# insert column using insert(position,column_name,first_column) function
sample_output.insert(0, 'Customer', first_column)
sample_output.style.set_caption("Sample Output for predicted WOOF Score").set_precision(3).background_gradient(cmap='Blues')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  """


,Customer,Alpha,Arf,Astro,Beam,Beethoven,Bezt,Bones,Choice,Flora,Garland,Hanover,Health One,Hearth,K99,Kastle,King,Omaha,Paws,Perro,Playtime,Rivera,Romero,Ruby,Seattle Gourmet,Top
0,1,0.284,0.002,0.000,0.000,0.003,0.440,0.000,0.000,0.011,0.000,0.125,0.094,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.006,0.020
1,2,0.675,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.223,0.000,0.014,0.000,0.073
2,3,0.289,0.000,0.000,0.000,0.000,0.514,0.000,0.000,0.001,0.000,0.089,0.054,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.016,0.012,0.025
